In [24]:
import numpy as np
import sklearn
from sklearn import model_selection, decomposition, datasets, linear_model
import matplotlib.pyplot as plt

In [25]:
import pandas as pd

In [26]:
iris_dataset = datasets.load_iris()

In [27]:
iris_df = pd.DataFrame(iris_dataset['data'], columns=iris_dataset['feature_names'])
iris_df['target'] = iris_dataset['target']
no_classes = len(set(iris_dataset['target']))

In [28]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [81]:
y = iris_dataset.target.reshape(-1,1)
X = iris_dataset.data

scaler = sklearn.preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X = np.insert(X, 0, 1, axis=1)

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y)
Y_ver = Y_test.T
oneHotEncoder = sklearn.preprocessing.OneHotEncoder()
Y_train = oneHotEncoder.fit_transform(Y_train)
Y_test = oneHotEncoder.transform(Y_test)

In [82]:
Y_ver

array([[0, 0, 0, 1, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 2, 1, 0, 2, 0, 1, 1,
        2, 2, 2, 1, 1, 2, 2, 1, 1, 0, 1, 1, 2, 1, 0, 0]])

In [83]:
def sigmoid(x): 
    return 1/(1+np.exp(-x))

In [84]:
def softmax(x):
    exp_mat = np.exp(x)
    return exp_mat / np.sum(exp_mat, axis=1, keepdims=True)

In [85]:
def softmax_regression(X, y, k, alpha, iterations):
    M, N = X.shape
    param_mat = np.zeros((k,N))
    for i in range(k):
        param_vec = np.random.rand(1,N)
        for j in range(iterations):
            batch_gradient = 0
            for z in range(M):
                y_pred = sigmoid(param_vec @ X[z])
                error = y[z,i] - y_pred
                gradient = error * X[z]
                batch_gradient += gradient
            param_vec += alpha * batch_gradient/M
        param_mat[i] = param_vec
    return param_mat

In [104]:
p = softmax_regression(X_train, Y_train,3, 0.01, 10000)

In [105]:
p

array([[-1.78248918e+00, -1.03951995e+00,  1.96937213e+00,
        -1.96439571e+00, -2.05777813e+00],
       [-1.07933471e+00,  6.88243053e-01, -1.60134426e+00,
         3.78277130e-01, -1.06707274e+00],
       [-2.88740317e+00, -2.44863400e-01,  7.08710949e-04,
         1.89235851e+00,  3.46519966e+00]])

In [107]:
def predict(X_test):
    M,N = X_test.shape
    predictions = np.zeros((3,M))
    for i in range(3):
        y_pred = sigmoid(X_test @ p[i].T)
        predictions[i] = y_pred
    predictions = np.argmax(predictions.T, axis=1)
    return predictions
    
y_pred = predict(X_test)
y_pred

array([0, 0, 0, 1, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 2, 2, 0, 2, 0, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 2, 1, 0, 0])

In [108]:
accuracy = sklearn.metrics.accuracy_score(Y_ver[0], y_pred)

In [109]:
accuracy

0.868421052631579

In [23]:
#softmax regression using sklearn
softmax_Reg = linear_model.LogisticRegression(multi_class="multinomial")
softmax_Reg.fit(X_train, Y_train)

C:\Users\susan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='multinomial')

In [24]:
y_softPred = softmax_Reg.predict(X_test)

In [25]:
accuracy = sklearn.metrics.accuracy_score(Y_test, y_softPred)

In [26]:
accuracy

1.0